In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import json
import os
from string import digits
import glob
import re

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

from molmass import Formula

from main_functions import *
from clone_ds_to_beta import copy_beta

# Molecule database merge with experimental MS/MS data for METASPACE.

This script merges a molecule databae with experimental MS/MS data.  These data can then be submitted to METASPACE to search for nuetral loss and in-source fragments within datasets.

See: /Users/dis/PycharmProjects/word2vec/main_functions.py

In [46]:
# Load reference database by path:
path = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'
ref_db = load_molecule_database(path)

In [4]:
# Parses GNPS_json to GNPS_df:
GNPS_json = '/Users/dis/PycharmProjects/word2vec/ALL_GNPS.json'
GNPS_df = parse_GNPS_json(GNPS_json)

In [5]:
# Needs structures to rescue 'PNNL-LIPIDS-POSITIVE'
# Drop for now by turning code line 173 off.
GNPS_df.library_membership.value_counts()

MONA                                               30784
MASSBANK                                            8729
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE     5725
GNPS-LIBRARY                                        2662
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE       1398
MASSBANKEU                                          1287
GNPS-NIH-NATURALPRODUCTSLIBRARY                     1266
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE     1041
RESPECT                                              981
SUMNER                                               795
GNPS-MSMLS                                           787
BILELIB19                                            747
GNPS-COLLECTIONS-PESTICIDES-POSITIVE                 644
GNPS-SELLECKCHEM-FDA-PART2                           592
GNPS-EMBL-MCF                                        585
CASMI                                                562
LDB_POSITIVE                                         461
GNPS-NIH-CLINICALCOLLECTION1   

In [6]:
# Identifies subset of GNPS with experimental specrta in ref db:
# Result: # 3920 spectra for 508 can_no_stereo_smiles
# Issue index 31383 when PNNL is included
GNPS_hits_df = search_GNPS_targets(ref_db, GNPS_df)

In [7]:
GNPS_hits_df.library_membership.value_counts()

MONA                                               1553
MASSBANK                                            794
GNPS-MSMLS                                          434
RESPECT                                             303
GNPS-EMBL-MCF                                       259
BILELIB19                                           191
GNPS-LIBRARY                                        111
SUMNER                                               95
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE        41
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE      25
GNPS-NIST14-MATCHES                                  19
MASSBANKEU                                           18
GNPS-COLLECTIONS-MISC                                18
CASMI                                                14
GNPS-NIH-CLINICALCOLLECTION1                         10
GNPS-SELLECKCHEM-FDA-PART2                           10
LDB_NEGATIVE                                          7
GNPS-NIH-NATURALPRODUCTSLIBRARY                 

In [8]:
GNPS_hits_df.Adduct.value_counts()

M+H        1982
M-H         923
M+Na        464
M+K         219
M+NH4+      176
M-H2O+H     135
M+NH4        13
M+            6
M+Cl          1
M-H2O-H       1
Name: Adduct, dtype: int64

In [9]:
# Identifies subset of Mona with experimental specrta in ref db, runs in a few hours
# Result: # 7646 spectra for 648 inchi matches
# Mona_hits_df = search_MONA(ref_db)

# Reload existing:
Mona_hits_df = pd.read_pickle('mona_2020_Apr_15.pickle')

# Filters Mona_hits_df for high-res instruments, good adducts, and <20 ppm error.
# Result: # 3437 spectra for 535 inchi matches
Mona_hits_df = parse_MONA_out(Mona_hits_df)

In [10]:
Mona_hits_df.adduct.value_counts()

M+H        2805
M+Na        254
M+K         153
M+NH4+      142
M-H2O+H      56
M+           27
Name: adduct, dtype: int64

In [11]:
Mona_hits_df.source.value_counts()

MassBank                                   1197
Vaniya/Fiehn Natural Products Library       836
unknown                                     471
Fiehn HILIC Library                         379
GNPS                                        296
ReSpect                                     199
RIKEN PlaSMA Authentic Standard Library      59
Name: source, dtype: int64

# Joinging HMDB predicted
3. Use to filter reference db on id
4. Repeat for theoretical of no expirimental

4. Copy files .txt to correct folder based on name

n. flag if hit found somehow? do not in of join...

In [41]:
rerun = False

#Import experimental and computational HMDB spectra, slow.
if rerun == True:
    hmdb_pos_exptl_df = parse_hmdb('/Users/dis/Desktop/hmdb_LCMS/hmdb_experimental_msms_spectra',
                               theo_flag=False)
    hmdb_pos_theo_df = parse_hmdb('/Users/dis/Desktop/hmdb_LCMS/hmdb_predicted_msms_spectra', 
                              theo_flag=True)
    
    hmdb_pos_exptl_df.to_pickle('hmdb_pos_exptl_df.pickle')
    hmdb_pos_theo_df.to_pickle('hmdb_pos_theo_df.pickle')

# Reload without reprocessing
else:
    hmdb_pos_exptl_df = pd.read_pickle('hmdb_pos_exptl_df.pickle')
    hmdb_pos_theo_df = pd.read_pickle('hmdb_pos_theo_df.pickle')

In [45]:
hmdb_pos_exptl_df.adduct.value_counts()

M+H    1371
Name: adduct, dtype: int64

In [13]:
# Cleans up ref db for only entries with experimental MS/MS spectra
# Without theoretical specta: 855 id's, with: 11440
ref_dfs = preparser_Sirius(ref_db,
                      GNPS_hits_df,
                      Mona_hits_df,
                      hmdb_pos_exptl_df,
                      hmdb_pos_theo_df)

expt_df = ref_dfs[0]
theo_df = ref_dfs[1]

In [23]:
# Small dataset to test
# expt_df = expt_df.iloc[0:25,:].copy(deep=True)
# theo_df = theo_df.iloc[0:25,:].copy(deep=True)

In [24]:
# Runs main Sirius loop
sirius_expt_df = loop_Sirius(expt_df, 
                               Mona_hits_df, 
                               GNPS_hits_df,
                               hmdb_pos_exptl_df,
                               hmdb_pos_theo_df.iloc[0:1,:])
sirius_expt_df.to_pickle('sirius_expt_df.pickle')


 series 0 0_HMDB0003361_Pyrimidine
0   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 5 5_HMDB0029713_Thiazole
5   M+H
m_df: 1 g_df 2 e_df 0 t_df 0

 series 15 15_HMDB0001873_Isobutyric_acid
15   M+H
m_df: 1 g_df 0 e_df 0 t_df 0

 series 16 16_HMDB0000039_Butyric_acid
16   M+H
m_df: 1 g_df 0 e_df 0 t_df 0

 series 17 17_HMDB0001414_Putrescine
17   M+H
m_df: 9 g_df 2 e_df 1 t_df 0

 series 18 18_HMDB0000056_betaAlanine
18   M+H
m_df: 6 g_df 2 e_df 1 t_df 0

 series 19 19_HMDB0000161_LAlanine
19   M+H
m_df: 4 g_df 5 e_df 1 t_df 0

 series 20 20_HMDB0001310_DAlanine
20   M+H
m_df: 0 g_df 5 e_df 0 t_df 0

 series 21 21_HMDB0000271_Sarcosine
21   M+H
m_df: 4 g_df 0 e_df 0 t_df 0

 series 23 23_HMDB0001311_DLactic_acid
23   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 24 24_HMDB0000190_LLactic_acid
24   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 25 25_HMDB0001051_Glyceraldehyde

 series 33 33_HMDB0001429_Phosphate
33   M+H
m_df: 3 g_df 0 e_df 0 t_df 0

 series 38 38_HMDB0000005_2Ketobutyric_acid
38 


 series 229 229_HMDB0006024_Mevalonolactone
229   M+K
m_df: 0 g_df 3 e_df 0 t_df 0
229   M+H
m_df: 0 g_df 3 e_df 0 t_df 0
229   M+Na
m_df: 0 g_df 3 e_df 0 t_df 0

 series 233 233_HMDB0000491_3Methyl2oxovaleric_acid

 series 251 251_HMDB0002064_NAcetylputrescine
251   M+H
m_df: 3 g_df 0 e_df 1 t_df 0

 series 252 252_HMDB0001432_Agmatine
252   M+H
m_df: 9 g_df 2 e_df 2 t_df 0

 series 258 258_HMDB0000766_NAcetylLalanine
258   M+H
m_df: 5 g_df 0 e_df 0 t_df 0

 series 259 259_HMDB0060460_cis4HydroxyDproline
259   M+H
m_df: 0 g_df 11 e_df 0 t_df 0
259   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 261 261_HMDB0001149_5Aminolevulinic_acid
261   M+H
m_df: 5 g_df 4 e_df 1 t_df 0

 series 262 262_HMDB0000783_Propionylglycine
262   M+H
m_df: 2 g_df 0 e_df 0 t_df 0

 series 263 263_HMDB0000725_4Hydroxyproline
263   M+H
m_df: 6 g_df 11 e_df 1 t_df 0
263   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 264 264_HMDB0000064_Creatine
264   M+H
m_df: 6 g_df 10 e_df 9 t_df 0
264   M+Na
m_df: 0 g_df 1 e_df 


 series 453 453_HMDB0000508_Ribitol
453   M+K
m_df: 0 g_df 6 e_df 0 t_df 0
453   M+H
m_df: 0 g_df 9 e_df 8 t_df 0
453   M+Na
m_df: 1 g_df 9 e_df 0 t_df 0

 series 454 454_HMDB0002917_DXylitol
454   M+K
m_df: 0 g_df 6 e_df 0 t_df 0
454   M+H
m_df: 0 g_df 9 e_df 0 t_df 0
454   M+Na
m_df: 0 g_df 9 e_df 0 t_df 0

 series 457 457_HMDB0060179_3SulfinatoLalaninate
457   M+H
m_df: 3 g_df 3 e_df 0 t_df 0

 series 458 458_HMDB0000996_3Sulfinoalanine
458   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 459 459_HMDB0001476_3Hydroxyanthranilic_acid
459   M+H
m_df: 6 g_df 2 e_df 1 t_df 0

 series 461 461_HMDB0000073_Dopamine
461   M+H
m_df: 22 g_df 10 e_df 0 t_df 0

 series 462 462_HMDB0013253_NAcetylhistamine_
462   M+H
m_df: 2 g_df 0 e_df 0 t_df 0

 series 463 463_HMDB0000152_Gentisic_acid
463   M+H
m_df: 3 g_df 3 e_df 4 t_df 0

 series 465 465_HMDB0000397_2Pyrocatechuic_acid
465   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
465   M+Na
m_df: 3 g_df 3 e_df 0 t_df 0

 series 477 477_HMDB0000177_LHistidine
477   M+

723   M+Na
m_df: 1 g_df 0 e_df 0 t_df 0

 series 724 724_HMDB0003416_DArginine
724   M+H
m_df: 0 g_df 8 e_df 0 t_df 0

 series 725 725_HMDB0004370_NMethyltryptamine
725   M+H
m_df: 3 g_df 4 e_df 0 t_df 0

 series 727 727_HMDB0000812_NAcetylLaspartic_acid
727   M+H
m_df: 4 g_df 0 e_df 0 t_df 0

 series 728 728_HMDB0003157_Guanidinosuccinic_acid
728   M+H
m_df: 3 g_df 1 e_df 0 t_df 0

 series 729 729_HMDB0000197_Indoleacetic_acid
729   M+H
m_df: 20 g_df 13 e_df 3 t_df 0
729   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 732 732_HMDB0000904_Citrulline
732   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
732   M+H
m_df: 15 g_df 6 e_df 9 t_df 0
732   M+Na
m_df: 3 g_df 3 e_df 0 t_df 0

 series 734 734_HMDB0000044_Ascorbic_acid
734   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
734   M+H
m_df: 4 g_df 4 e_df 0 t_df 0
734   M+Na
m_df: 3 g_df 3 e_df 0 t_df 0

 series 735 735_HMDB0006355_DGlucurono63lactone
735   M+Na
m_df: 0 g_df 1 e_df 0 t_df 0

 series 736 736_HMDB0000402_2Isopropylmalic_acid
736   M+H
m_df: 0 g_df 1 e_d


 series 915 915_HMDB0029965_Methyl_betaDglucopyranoside
915   M+Na
m_df: 0 g_df 1 e_df 0 t_df 0

 series 922 922_HMDB0000625_Gluconic_acid
922   M+H
m_df: 0 g_df 2 e_df 0 t_df 0
922   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 923 923_HMDB0000565_Galactonic_acid
923   M+H
m_df: 0 g_df 2 e_df 0 t_df 0
923   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 924 924_HMDB0003290_Gulonic_acid
924   M+H
m_df: 0 g_df 2 e_df 0 t_df 0
924   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 931 931_HMDB0035157_Geranyl_acetate
931   M+H
m_df: 1 g_df 0 e_df 0 t_df 0

 series 937 937_HMDB0003966_Selenomethionine
937   M+H
m_df: 1 g_df 2 e_df 0 t_df 0

 series 939 939_HMDB0000609_DLDopa
939   M+K
m_df: 0 g_df 3 e_df 0 t_df 0
939   M+H
m_df: 5 g_df 13 e_df 0 t_df 0
939   M+Na
m_df: 0 g_df 3 e_df 0 t_df 0

 series 942 942_HMDB0000291_Vanillylmandelic_acid
942   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 947 947_HMDB0034160_Citronellyl_acetate
947   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
947   M+H
m_df: 3 g_df 3 e_df 0 t_

...


 series 1612 1612_HMDB0000053_Androstenedione
1612   M+H
m_df: 11 g_df 2 e_df 0 t_df 0

 series 1626 1626_HMDB0000791_LOctanoylcarnitine
1626   M+H
m_df: 3 g_df 0 e_df 0 t_df 0

 series 1627 1627_HMDB0032358_Lauroyl_diethanolamide
1627   M+H
m_df: 7 g_df 2 e_df 0 t_df 0

 series 1629 1629_HMDB0000431_1617Epiestriol

 series 1630 1630_HMDB0000153_Estriol
1630   M+H
m_df: 2 g_df 0 e_df 0 t_df 0

 series 1631 1631_HMDB0000347_16bHydroxyestradiol

 series 1633 1633_HMDB0000899_Androstanedione
1633   M+H
m_df: 3 g_df 3 e_df 0 t_df 0
1633   M+Na
m_df: 3 g_df 3 e_df 0 t_df 0

 series 1634 1634_HMDB0000077_Dehydroepiandrosterone
1634   M+H
m_df: 9 g_df 2 e_df 1 t_df 0

 series 1635 1635_HMDB0000234_Testosterone
1635   M+H
m_df: 12 g_df 6 e_df 0 t_df 0
1635   M+Na
m_df: 2 g_df 0 e_df 0 t_df 0

 series 1636 1636_HMDB0000628_Epitestosterone
1636   M+H
m_df: 7 g_df 6 e_df 0 t_df 0

 series 1637 1637_HMDB0003769_Etiocholanedione_
1637   M+H
m_df: 0 g_df 3 e_df 0 t_df 0
1637   M+Na
m_df: 0 g_df 3 e


 series 1946 1946_HMDB0002183_Docosahexaenoic_acid
1946   M+H
m_df: 3 g_df 0 e_df 0 t_df 0

 series 1947 1947_HMDB0011565_MG1619Z0000
1947   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 1948 1948_HMDB0031043_Avocadene_1acetate
1948   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
1948   M+H
m_df: 4 g_df 4 e_df 0 t_df 0
1948   M+Na
m_df: 5 g_df 3 e_df 0 t_df 0

 series 1949 1949_HMDB0031044_Avocadene_2acetate
1949   M+H
m_df: 1 g_df 1 e_df 0 t_df 0

 series 1950 1950_HMDB0031045_Avocadene_4acetate
1950   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
1950   M+H
m_df: 4 g_df 4 e_df 0 t_df 0
1950   M+Na
m_df: 3 g_df 3 e_df 0 t_df 0

 series 1956 1956_HMDB0000058_Cyclic_AMP
1956   M+H
m_df: 14 g_df 15 e_df 10 t_df 0
1956   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 1957 1957_HMDB0011616_Adenosine_23cyclic_phosphate
1957   M+H
m_df: 0 g_df 1 e_df 0 t_df 0
1957   M+Na
m_df: 0 g_df 1 e_df 0 t_df 0

 series 1959 1959_HMDB0003601_SReticuline
1959   M+H
m_df: 0 g_df 3 e_df 0 t_df 0

 series 1966 1966_HMDB0000016_Deoxycortico

2352   M+H
m_df: 0 g_df 3 e_df 0 t_df 0
2352   M+Na
m_df: 0 g_df 3 e_df 0 t_df 0

 series 2354 2354_HMDB0000067_Cholesterol
2354   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
2354   M+H
m_df: 3 g_df 8 e_df 0 t_df 0
2354   M+Na
m_df: 3 g_df 3 e_df 0 t_df 0

 series 2356 2356_HMDB0011182_5betaCholestanone
2356   M+K
m_df: 0 g_df 3 e_df 0 t_df 0
2356   M+H
m_df: 0 g_df 3 e_df 0 t_df 0
2356   M+Na
m_df: 0 g_df 3 e_df 0 t_df 0

 series 2358 2358_HMDB0001245_dCDP
2358   M+H
m_df: 4 g_df 0 e_df 0 t_df 0

 series 2362 2362_HMDB0001569_Epicoprostanol
2362   M+K
m_df: 0 g_df 3 e_df 0 t_df 0
2362   M+H
m_df: 0 g_df 3 e_df 0 t_df 0

 series 2363 2363_HMDB0000577_5betaCoprostanol
2363   M+K
m_df: 0 g_df 3 e_df 0 t_df 0
2363   M+H
m_df: 0 g_df 3 e_df 0 t_df 0

 series 2364 2364_HMDB0000908_5alphaCholestanol
2364   M+K
m_df: 3 g_df 3 e_df 0 t_df 0
2364   M+H
m_df: 3 g_df 3 e_df 0 t_df 0

 series 2366 2366_HMDB0000280_Phosphoribosyl_pyrophosphate

 series 2370 2370_HMDB0000503_7alphaHydroxy3oxo5betacholan24oic_a


 series 2529 2529_HMDB0001457_5bCholestane3a_7a_12atriol
2529   M+Na
m_df: 1 g_df 1 e_df 0 t_df 0

 series 2533 2533_HMDB0001124_Trehalose_6phosphate
2533   M+H
m_df: 1 g_df 0 e_df 0 t_df 0

 series 2537 2537_HMDB0006461_Linoelaidyl_carnitine
2537   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 2538 2538_HMDB0006469_Linoleyl_carnitine
2538   M+H
m_df: 0 g_df 1 e_df 0 t_df 0

 series 2542 2542_HMDB0000437_3a4b7a12aTetrahydroxy5bcholanoic_acid
2542   M+H
m_df: 1 g_df 1 e_df 0 t_df 0
2542   M+Na
m_df: 1 g_df 1 e_df 0 t_df 0

 series 2543 2543_HMDB0000399_3a6b7a12aTetrahydroxy5bcholanoic_acid
2543   M+H
m_df: 0 g_df 3 e_df 0 t_df 0
2543   M+Na
m_df: 0 g_df 9 e_df 0 t_df 0

 series 2545 2545_HMDB0000377_3a6b7b12bTetrahydroxy5bcholanoic_acid
2545   M+H
m_df: 0 g_df 3 e_df 0 t_df 0
2545   M+Na
m_df: 0 g_df 9 e_df 0 t_df 0

 series 2546 2546_HMDB0000307_1bHydroxycholic_acid
2546   M+Na
m_df: 1 g_df 1 e_df 0 t_df 0

 series 2547 2547_HMDB0000367_3a6b7b12aTetrahydroxy5bcholanoic_acid
2547   M+H
m_d


 series 3562 3562_HMDB0008824_PE140160
3562   M+H
m_df: 1 g_df 1 e_df 0 t_df 0

 series 3567 3567_HMDB0000902_NAD
3567   M+
m_df: 5 g_df 0 e_df 0 t_df 0
3567   M+H
m_df: 0 g_df 0 e_df 8 t_df 0

 series 3577 3577_HMDB0001487_NADH
3577   M+H
m_df: 1 g_df 0 e_df 0 t_df 0

 series 3579 3579_HMDB0003553_Stachyose
3579   M+Na
m_df: 0 g_df 2 e_df 0 t_df 0

 series 3745 3745_HMDB0007859_PA1601819Z
3745   M+H
m_df: 0 g_df 1 e_df 0 t_df 0
3745   M+Na
m_df: 0 g_df 1 e_df 0 t_df 0

 series 3780 3780_HMDB0008890_PE150160
3780   M+H
m_df: 1 g_df 1 e_df 0 t_df 0

 series 3790 3790_HMDB0012330_PS140140

 series 3840 3840_HMDB0008924_PE1601619Z
3840   M+K
m_df: 1 g_df 1 e_df 0 t_df 0
3840   M+H
m_df: 0 g_df 0 e_df 2 t_df 0
3840   M+Na
m_df: 1 g_df 1 e_df 0 t_df 0

 series 3956 3956_HMDB0004971_Glucosylceramide_d181160
3956   M+Na
m_df: 0 g_df 1 e_df 0 t_df 0

 series 4053 4053_HMDB0008963_PE1619Z1839Z12Z15Z
4053   M+Na
m_df: 0 g_df 1 e_df 0 t_df 0

 series 4065 4065_HMDB0008961_PE1619Z1829Z12Z
4065   

In [25]:
# Runs main Sirius loop
sirius_theo_df = loop_Sirius(theo_df, 
                               Mona_hits_df.iloc[0:1,:], 
                               GNPS_hits_df.iloc[0:1,:],
                               hmdb_pos_exptl_df.iloc[0:1,:],
                               hmdb_pos_theo_df)
sirius_theo_df.to_pickle('sirius_theo_df.pickle')


 series 1 1_HMDB0000240_Sulfite
1   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 2 2_HMDB0094691_Furanone_A
2   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 3 3_HMDB0001184_Methyl_propenyl_ketone
3   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 4 4_HMDB0012157_3Methyl2butenal
4   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 6 6_HMDB0002039_2Pyrrolidinone
6   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 7 7_HMDB0000549_GammaButyrolactone
7   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 8 8_HMDB0034439_Isocrotonic_acid
8   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 9 9_HMDB0003407_Diacetyl
9   M+H
m_df: 0 g_df 0 e_df 0 t_df 6

 series 10 10_HMDB0010720_But2enoic_acid
10   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11 11_HMDB0006478_IsoValeraldehyde
11   M+H
m_df: 0 g_df 0 e_df 0 t_df 6

 series 13 13_HMDB0000243_Pyruvic_acid
13   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 14 14_HMDB0011111_Malonic_semialdehyde
14   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 22 22_HMDB0002329_Oxalic_acid
22   M+H
m_df: 0 g_df


 series 185 185_HMDB0061945_2Ethyl2hexenal
185   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 186 186_HMDB0030966_3xi5Z15Octadien3ol
186   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 187 187_HMDB0030961_2Octenal
187   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 188 188_HMDB0013809_E2octenal
188   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 189 189_HMDB0031270_1Nonene
189   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 190 190_HMDB0062558_Pyroglutamine
190   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 192 192_HMDB0033793_4Heptenoic_acid
192   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 193 193_HMDB0031476_23Heptanedione
193   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 194 194_HMDB0031504_Methyl_Z3hexenoate
194   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 195 195_HMDB0031484_2Heptenoic_acid
195   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 196 196_HMDB0031681_Dihydro5propyl23Hfuranone
196   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 197 197_HMDB0031603_Ethyl_4pentenoate
197   M+H
m_df: 0 g_df 0 e_df 0 t_df 3


 series 340 340_HMDB0000469_5Hydroxymethyluracil
340   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 341 341_HMDB0036236_2Methylallyl_butyrate
341   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 342 342_HMDB0032164_Allyl_valerate
342   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 343 343_HMDB0039793_E4Octenoic_acid
343   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 344 344_HMDB0000392_2Octenoic_acid
344   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 345 345_HMDB0029761_Ethyl_2methyl4pentenoate
345   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 346 346_HMDB0001568_trans2Octenoic_acid
346   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 347 347_HMDB0032266_Ethyl_E3hexenoate
347   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 348 348_HMDB0032207_cis5Octenoic_acid
348   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 349 349_HMDB0040581_Z6Nonen1ol
349   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 350 350_HMDB0037304_26Dimethyl6hepten1ol
350   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 351 351_HMDB0034152_1Nonen3ol
351   M+H
m


 series 488 488_HMDB0039854_1Decen3ol
488   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 489 489_HMDB0032206_cis4Decenol
489   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 490 490_HMDB0032212_3Decanone
490   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 491 491_HMDB0031409_2Decanone
491   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 492 492_HMDB0032532_2DECENOL
492   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 493 493_HMDB0013810_E3decen1ol
493   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 494 494_HMDB0031445_Undecane
494   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 495 495_HMDB0037267_xi3Methyldecane
495   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 496 496_HMDB0037268_xi4Methyldecane
496   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 497 497_HMDB0039000_2Methyldecane
497   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 498 498_HMDB0001241_2Aminomuconic_acid
498   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 503 503_HMDB0037759_2Methylene4oxopentanedioic_acid
503   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 504 504


 series 629 629_HMDB0032312_24Hexadienyl_butyrate
629   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 630 630_HMDB0040247_2Undecenal
630   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 631 631_HMDB0033577_10Undecen2one
631   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 632 632_HMDB0031128_10Undecenal
632   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 633 633_HMDB0061820_Z3Dodecene
633   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 634 634_HMDB0094667_3Methyl2undecene_Z
634   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 635 635_HMDB0059874_1Dodecene
635   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 636 636_HMDB0094665_22466Pentamethyl3heptene
636   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 637 637_HMDB0061821_E3Dodecene
637   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 638 638_HMDB0094666_Nonyl_cyclopropane
638   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 639 639_HMDB0061819_E2Dodecene
639   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 640 640_HMDB0003288_Selenocysteine
640   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 se

...


 series 10906 10906_HMDB0051855_TG22113Z2055Z8Z11Z14Z17Z203n6
10906   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10907 10907_HMDB0046920_TG2202055Z8Z11Z14Z17Z2045Z8Z11Z14Z
10907   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10908 10908_HMDB0054822_TG2247Z10Z13Z16Z22213Z16Z1839Z12Z15Z
10908   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10909 10909_HMDB0046745_TG2202045Z8Z11Z14Z2045Z8Z11Z14Z
10909   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10910 10910_HMDB0054688_TG2247Z10Z13Z16Z2201846Z9Z12Z15Z
10910   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10911 10911_HMDB0051736_TG22113Z2045Z8Z11Z14Z203n6
10911   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10912 10912_HMDB0046919_TG2202055Z8Z11Z14Z17Z203n6
10912   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10913 10913_HMDB0054545_TG22213Z16Z22213Z16Z1846Z9Z12Z15Z
10913   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10914 10914_HMDB0051854_TG22113Z2055Z8Z11Z14Z17Z202n6
10914   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10915 10915_HMDB0054759_TG2247Z10Z13Z16Z


 series 10994 10994_HMDB0068970_TG220220200
10994   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10995 10995_HMDB0069245_TG220210210
10995   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10996 10996_HMDB0062633_Trans2octadecenoylCoA
10996   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10997 10997_HMDB0001114_StearoylCoA
10997   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 10998 10998_HMDB0054829_TG2247Z10Z13Z16Z2247Z10Z13Z16Z2247Z10Z13Z16Z
10998   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11000 11000_HMDB0069163_TG220220210
11000   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11002 11002_HMDB0054541_TG22213Z16Z22213Z16Z22213Z16Z
11002   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11005 11005_HMDB0001165_Keratan
11005   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11006 11006_HMDB0051629_TG22113Z22113Z22113Z
11006   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11007 11007_HMDB0001980_Vasopressin
11007   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11008 11008_HMDB0046381_TG220220220
11008   M+H
m_df: 0 g_df 0 e_


 series 11137 11137_HMDB0056713_CL1601829Z12Z1829Z12Z180
11137   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11138 11138_HMDB0056740_CL1602045Z8Z11Z14Z180160
11138   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11139 11139_HMDB0056700_CL1601829Z12Z1801829Z12Z
11139   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11141 11141_HMDB0056710_CL1601829Z12Z1819Z1819Z
11141   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11143 11143_HMDB0058326_CL1819Z1601819Z1829Z12Z
11143   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11144 11144_HMDB0056946_CL1801602045Z8Z11Z14Z160
11144   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11146 11146_HMDB0058333_CL1819Z1601829Z12Z1819Z
11146   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11147 11147_HMDB0058695_CL1829Z12Z1601829Z12Z180
11147   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11148 11148_HMDB0056520_CL1601802045Z8Z11Z14Z160
11148   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11149 11149_HMDB0056666_CL1601819Z1819Z1829Z12Z
11149   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11150


 series 11273 11273_HMDB0056521_CL1601802045Z8Z11Z14Z180
11273   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11275 11275_HMDB0057146_CL1801819Z1801829Z12Z
11275   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11276 11276_HMDB0058362_CL1819Z1801829Z12Z180
11276   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11277 11277_HMDB0057209_CL1801829Z12Z1819Z180
11277   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11278 11278_HMDB0056994_CL1801801819Z1829Z12Z
11278   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11279 11279_HMDB0057175_CL1801819Z1829Z12Z180
11279   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11280 11280_HMDB0058356_CL1819Z1801819Z1819Z
11280   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11281 11281_HMDB0057169_CL1801819Z1819Z1819Z
11281   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11282 11282_HMDB0057003_CL1801801829Z12Z1819Z
11282   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11283 11283_HMDB0056993_CL1801801819Z1819Z
11283   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11284 11284_HMDB0057000_CL1801801


 series 11377 11377_HMDB0056965_CL1801801802045Z8Z11Z14Z
11377   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11378 11378_HMDB0004944_Ganglioside_GM2_d181260
11378   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11379 11379_HMDB0059063_CL2045Z8Z11Z14Z1829Z12Z2045Z8Z11Z14Z1829Z12Z
11379   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11381 11381_HMDB0056763_CL1602045Z8Z11Z14Z2045Z8Z11Z14Z2045Z8Z11Z14Z
11381   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11384 11384_HMDB0058864_CL1829Z12Z2045Z8Z11Z14Z2045Z8Z11Z14Z1829Z12Z
11384   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11386 11386_HMDB0058855_CL1829Z12Z2045Z8Z11Z14Z1829Z12Z2045Z8Z11Z14Z
11386   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11388 11388_HMDB0058968_CL2045Z8Z11Z14Z1602045Z8Z11Z14Z2045Z8Z11Z14Z
11388   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11390 11390_HMDB0058543_CL1819Z1829Z12Z2045Z8Z11Z14Z2045Z8Z11Z14Z
11390   M+H
m_df: 0 g_df 0 e_df 0 t_df 3

 series 11391 11391_HMDB0058863_CL1829Z12Z2045Z8Z11Z14Z2045Z8Z11Z14Z1819Z
11391   M+H
m_df:

In [65]:
# Load dataframes and merge, check for duplicates
# 9121 theo + 781 expt= 9902
sirius_expt_df = pd.read_pickle('sirius_expt_df.pickle')
sirius_expt_df['source'] = 'expt'
sirius_theo_df = pd.read_pickle('sirius_theo_df.pickle')
sirius_theo_df['source'] = 'theo'
sirius_merge_df = pd.concat([sirius_theo_df, sirius_expt_df], sort=True)
sirius_merge_df = sirius_merge_df[['exists', 'source']].copy(deep=True)
sirius_merge_df.to_pickle('sirius_merge_df.pickle')

In [66]:
sirius_merge_df.source.value_counts()

theo    9121
expt     781
Name: source, dtype: int64

In [68]:
sirius_merge_df.iloc[0,:].exists

'/Users/dis/PycharmProjects/word2vec/trees/1_HMDB0000240_Sulfite/1_HMDB0000240_Sulfite/1_HMDB0000240_ms_ms_spectrum_23546_predicted_/spectra/1_H2O3S_M+H+.ms'

In [69]:
# Reload without rerunning
sirius_merge_df = pd.read_pickle('sirius_merge_df.pickle')

In [70]:
# Clean-up and merge output with MS2 spectra
ref_df = pd.concat([ref_dfs[0], ref_dfs[1]], sort=True)

In [74]:
# Clean-up and merge output with MS2 spectra
clean_output_df = results_clean_up(ref_df, sirius_merge_df)

df_merge:  500
df_merge:  1000
df_merge:  1500
df_merge:  2000
df_merge:  2500
df_merge:  3000
df_merge:  3500
df_merge:  4000
df_merge:  4500
df_merge:  5000
df_merge:  5500
df_merge:  6000
df_merge:  6500
df_merge:  7000
df_merge:  7500
df_merge:  8000
df_merge:  8500
df_merge:  9000
df_merge:  9500
Observed ions:  1     193494
22      3046
38         6
0          1
Name: dmass, dtype: int64
Correct ionformulas:  True    196514
Name: good_mass_calc, dtype: int64


In [99]:
# Clean-up database for METASPACE
METASPACE_db = output_METASPACE(clean_output_df)

In [100]:
# Export entire db
METASPACE_db.iloc[:,0:4].to_csv('out_db/core_metabolome_v3_msms_2020_Apr_28.csv', sep='\t')

In [ ]:
### Still to copy to new notebook/scripts below:

In [115]:
# Filter DB for observed MS1 and export as custom databases
# Loop over all 10x wholebody datasets:
# Run: 1) entire DB (above), 2) each (this cell), 3) consensus 10x merge after
from core_met_v3_20FDR_wb_results import *

any_ds_ids = []
for result in cmv3_wb_result_paths:
    ids = list(pd.read_csv(result, header=2).moleculeIds) # what row to start?
    ids = str(ids)
    ids = ids.replace("'", "")
    ids = ids.replace(" ", "")
    ids = ids.replace("[", "")
    ids = ids.replace("]", "")
    ids = ids.split(",")
    this_db = METASPACE_db[METASPACE_db.old_id.isin(ids)]
    
    out = result.rsplit('/',)[-1]
    out_name = 'out_db/' + out
    any_ds_ids = any_ds_ids + ids
    this_db.iloc[:,0:4].to_csv(out_name, sep='\t')
    print(out, ':', this_db.shape[0])

any_df = METASPACE_db[METASPACE_db.old_id.isin(any_ds_ids)]
any_df.iloc[:,0:4].to_csv('out_db/any_ds_db_msms_2020_Apr_28.csv', 
                          sep='\t')
print('any_df_idd', ':', any_df.shape[0])
print('cm_v3_MSMS', ':', METASPACE_db.shape[0])

2016-09-21_16h07m45s_Technologie_ServierTES-WBrat-vehicle.csv : 4409
2017-05-17_19h49m04s_whole body xenograft (1) [RMS norm].csv : 13116
2017-05-17_19h50m07s_wb xenograft trp pathway dosed- rms_corrected.csv : 13420
2017-05-29_07h28m52s_servier_TT_mouse_wb_fmpts_derivatization_CHCA.csv : 2704
2017-07-24_19h42m31s_Servier_Ctrl_mouse_wb_lateral_plane_9aa.csv : 3776
2017-07-26_18h25m14s_Servier_Ctrl_mouse_wb_median_plane_9aa.csv : 5405
2017-08-03_15h09m06s_Servier_Ctrl_mouse_wb_median_plane_chca.csv : 9271
2017-08-03_15h09m51s_Servier_Ctrl_mouse_wb_lateral_plane_chca.csv : 10767
2017-08-11_07h59m58s_Servier_Ctrl_mouse_wb_lateral_plane_DHB.csv : 11051
2017-08-11_08h01m02s_Servier_Ctrl_mouse_wb_median_plane_DHB.csv : 12403
any_df_idd : 37027
cm_v3_MSMS : 196514


In [119]:
# Copies datsets to beta!
# Run succesfully.
ds_to_copy_beta_list = list(cmv3_wb_ds_name_dict.keys())
for ds in ds_to_copy_beta_list:
    copy_beta(ds, 'HNaKM', 'core_metabolome_v3')

In [136]:
# Copies datsets to beta!
# Make function work for specific, all, any
# Running!
for ds in ds_to_copy_beta_list:
    copy_beta(ds, 'M', 'core_metabolome_v3_msms_2020_Apr_28')
# Running!
for ds in ds_to_copy_beta_list:
    copy_beta(ds, 'M', 'any_ds_db_msms_2020_Apr_28')

In [140]:
beta_ds_db_dict = {'2016-09-21_16h07m45s': '2016-09-21_16h07m45s_Technologie_ServierTES-WBrat-vehicle',
                   '2017-05-17_19h49m04s': '2017-05-17_19h49m04s_whole_body_xenograft_(1)_[RMS_norm]',
                   '2017-05-17_19h50m07s': '2017-05-17_19h50m07s_wb_xenograft_trp_pathway_dosed-_rms_corrected',
                   '2017-05-29_07h28m52s': '2017-05-29_07h28m52s_servier_TT_mouse_wb_fmpts_derivatization_CHCA',
                   '2017-07-24_19h42m31s': '2017-07-24_19h42m31s_Servier_Ctrl_mouse_wb_lateral_plane_9aa',
                   '2017-07-26_18h25m14s': '2017-07-26_18h25m14s_Servier_Ctrl_mouse_wb_median_plane_9aa',
                   '2017-08-03_15h09m06s': '2017-08-03_15h09m06s_Servier_Ctrl_mouse_wb_median_plane_chca',
                   '2017-08-03_15h09m51s': '2017-08-03_15h09m51s_Servier_Ctrl_mouse_wb_lateral_plane_chca',
                   '2017-08-11_07h59m58s': '2017-08-11_07h59m58s_Servier_Ctrl_mouse_wb_lateral_plane_DHB',
                   '2017-08-11_08h01m02s': '2017-08-11_08h01m02s_Servier_Ctrl_mouse_wb_median_plane_DHB'}
# Running!
for ds, db in beta_ds_db_dict.items():
    copy_beta(ds, 'M', db)

Local plotting here:
http://localhost:8888/notebooks/PycharmProjects/word2vec/IMS_plotter_v2.ipynb

Next Jupyter notebook extract data from METASPACE:

http://localhost:8888/notebooks/PycharmProjects/word2vec/export_ion_mages.ipynb

http://localhost:8888/notebooks/PycharmProjects/word2vec/export_ion_images_batch.ipynb

Next Jupyter notebook extract data locally:

http://localhost:8888/notebooks/PycharmProjects/word2vec/IMS_plotter_v2.ipynb

In [15]:
## Reprocess for Veronika's spotted lipid dataset: (2020_May_04)

# Reload without rerunning
sirius_merge_df = pd.read_pickle('sirius_merge_df.pickle')

# Clean-up and merge output with MS2 spectra
ref_df = pd.concat([ref_dfs[0], ref_dfs[1]], sort=True) # Get ref df?

# Clean-up and merge output with MS2 spectra
clean_output_df = results_clean_up(ref_df, sirius_merge_df)

# Clean-up database for METASPACE
METASPACE_db = output_METASPACE(clean_output_df)

df_merge:  500
df_merge:  1000
df_merge:  1500
df_merge:  2000
df_merge:  2500
df_merge:  3000
df_merge:  3500
df_merge:  4000
df_merge:  4500
df_merge:  5000
df_merge:  5500
df_merge:  6000
df_merge:  6500
df_merge:  7000
df_merge:  7500
df_merge:  8000
df_merge:  8500
df_merge:  9000
df_merge:  9500
Observed ions: 
 1     193494
22      3046
38         6
0          1
Name: dmass, dtype: int64
Correct ionformulas: 
 True    196514
Name: good_mass_calc, dtype: int64


In [25]:
# Known compounds spotted:
lipids_spotted = {
    'TG(18:0/18:0/18:0)': 'HMDB0005393', 
    'TG(16:1/16:1/16:1)': 'HMDB0005432', 
    'LPC(16:0)': 'HMDB0010407', 
    'PA(18:1/18:1)': 'HMDB0007865', 
    'SM(d18:1/16:0)': 'HMDB0010169', 
    'PE(22:1/22:1)': 'HMDB0009535', 
    'PC(22:1/22:1)': 'HMDB0008578',
    'CE(18:2)':'HMDB0000610', 
    '6 Di-Sulfo, Galactosyl(ß)Ceramide(d18:1/12:0)': 'nd',
    'PI(17:0/17:0)': 'nd', 
    'PE(17:0/17:0)': 'nd', 
    'CholesterolPE(18:0/18:0)': 'nd',
    'Ergosterol': 'HMDB0000878'
}

lipids_spotted = list(lipids_spotted.values())
lipids_spotted = [x for x in lipids_spotted if x != 'nd']
lipids_spotted

['HMDB0005393',
 'HMDB0005432',
 'HMDB0010407',
 'HMDB0007865',
 'HMDB0010169',
 'HMDB0009535',
 'HMDB0008578',
 'HMDB0000610',
 'HMDB0000878']

In [31]:
# Filters
# 2020-03-12_17h55m21s
lipid_spot_df = METASPACE_db[METASPACE_db.old_id.isin(lipids_spotted)]
lipid_spot_df.iloc[:,0:4].to_csv('out_db/ds_2020-03-12_17h55m21s_MS2.csv', 
                          sep='\t')
lipid_spot_df.old_id.value_counts()

HMDB0000878    52
HMDB0000610    51
HMDB0010407    47
HMDB0007865    39
HMDB0005393    16
HMDB0008578    10
HMDB0005432     2
Name: old_id, dtype: int64

In [36]:
# Load reference database by path:
path = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'
ref_db = load_molecule_database(path)
lipid_spot_ms1_df = ref_db[ref_db.id.isin(lipids_spotted)]
lipid_spot_ms1_df.iloc[:,0:4].to_csv('out_db/ds_2020-03-12_17h55m21s_MS1.csv', 
                          sep='\t')
lipid_spot_ms1_df

,id,name,formula,inchi
2404,HMDB0000878,Ergosterol,C28H44O,InChI=1S/C28H44O/c1-18(2)19(3)7-8-20(4)24-11-1...
2741,HMDB0010407,LysoPC(P-16:0),C24H50NO6P,InChI=1S/C24H50NO6P/c1-5-6-7-8-9-10-11-12-13-1...
3487,HMDB0000610,"CE(18:2(9Z,12Z))",C45H76O2,InChI=1S/C45H76O2/c1-7-8-9-10-11-12-13-14-15-1...
3967,HMDB0007865,PA(18:1(9Z)/18:1(9Z)),C39H73O8P,InChI=1S/C39H73O8P/c1-3-5-7-9-11-13-15-17-19-2...
4003,HMDB0010169,SM(d18:1/16:0),C39H79N2O6P,InChI=1S/C39H79N2O6P/c1-6-8-10-12-14-16-18-20-...
5585,HMDB0005432,TG(16:1(9Z)/16:1(9Z)/16:1(9Z)),C51H92O6,InChI=1S/C51H92O6/c1-4-7-10-13-16-19-22-25-28-...
7605,HMDB0005393,TG(18:0/18:0/18:0),C57H110O6,InChI=1S/C57H110O6/c1-4-7-10-13-16-19-22-25-28...
7658,HMDB0008578,PC(22:1(13Z)/22:1(13Z)),C52H100NO8P,InChI=1S/C52H100NO8P/c1-6-8-10-12-14-16-18-20-...


In [40]:
# Copy to Beta:
ds = '2020-03-12_17h55m21s'
# copy_beta(ds, 'M', 'core_metabolome_v3_msms_2020_Apr_28')
copy_beta(ds, 'HNaKM', 'ds_2020-03-12_17h55m21s_MS1')
copy_beta(ds, 'M', 'ds_2020-03-12_17h55m21s_MS2')

#Then:
next_nb = 'http://localhost:8888/notebooks/PycharmProjects/word2vec/export_ion_images_batch.ipynb'

For positive dataset:
1. 13 total compounds spotted.
2. 9 in HMDB
3. 7 in cm_v3 with MSMS and exported
4. 2-52 ions per compound.